# Problema 1 (5 puntos)

Define una función `informe_nucleotidos(archivo_fasta, porciones, alfabeto)` que calcule el porcentaje de nucleótidos en cada porción de cada secuencia almacenada en un archivo fasta dado. 

Por ejemplo, si se le pasa `porciones=4` y `alfabeto=["A", "C", "G", "T"]`, la función encontrará, para cada secuencia, el porcentaje de nucleótidos en el primer, segundo, tercer y cuarto cuarto de cada secuencia. 
Análogamente, si se pasase `porciones=5`, obtendría los porcentajes para el primer, segundo, tercer, cuarto, y quinto quintos, etc. 

Al terminar el cálculo, la función:
1. __Escribirá__ a un segundo fichero un informe de nombre `informe_<archivo_fasta>.txt` con el siguiente formato (ejemplo para `porciones=4`):

```
secuencia_1 
A: [Ap11, Ap12, Ap13, Ap14]
C: [Cp11, Cp12, Cp13, Cp14]
T: [Tp11, Tp12, Tp13, Tp14]
G: [Gp11, Gp21, Gp31, Gp41]

secuencia_2
A: [Ap21, Ap22, Ap23, Ap4]
C: [Cp21, Cp22, Cp23, Cp4]
T: [Tp21, Tp22, Tp23, Tp4]
G: [Gp21, Gp22, Gp23, Gp4]

...

secuencia_n
A: [Apn1, Apn2, Apn3, Apn4]
C: [Cpn1, Cpn2, Cpn3, Cpn4]
T: [Tpn1, Tpn2, Tpn3, Tpn4]
G: [Gpn1, Gpn2, Gpn3, Gpn4]
```

Donde los valores {A,C,T,G}pmn son los porcentajes de cada nucleótido encontrados en cada porción para cada una de las secuencias, donde m = número de la secuencia, y n=ordinal de la porción a la que hace referencia el porcentaje: por ej.; primero cuarto = 1, segundo cuarto = 2, tercer cuarto = 3, cuarto cuarto = 4. 

2. __Devolverá__ una lista de listas con los porcentajes en el formato indicado en el anterior párrafo. 


_Nota 1_: Prueba tú codigo con los ficheros `../materiales-bio-python/labs/oric.fasta` y `../materiales-bio-python/labs/covid-samples.fasta`

_Nota 2_: Sí Python no encuentra alguno de los ficheros, prueba a resincronizar! 

_Nota 3_: Algunos ficheros FASTA contienen saltos de línea (`\n`): has de eliminarlos! 

In [ ]:
def calcula_frecuencia(nucleotido, porcion):
    """Calcula la frecuencia del nucleótido en la secuencia (porción).
    """
    return 100 * porcion.count(nucleotido) / len(porcion)

def parte_secuencia(sec, n, redondeo='floor'):
    """Divide una secuencia en n partes. Admite dos métodos distintos de redondeo.
    floor es el redondeo "de toda la vida" 
    closest redondea al entero más cercano. Este método hace que los tamaños de las porciones 
    sean lo más iguales posible.
    """
    porciones = []
    print (len(sec), n)
    if redondeo == 'floor':
        porcion_len = int(len(sec) / n) # Redondeamos al entero que resulta de eliminar la parte decimal.
    else:
        porcion_len = int(1.0 * len(sec) / n + 0.5) # Redondeamos al entero más cercano (intervalo cerrado)
        # porcion_len = round(len(sec) / n) # Podría usar esto, pero también tendría problemas...
    print("porcion_len es %d" % porcion_len)
    print()
    for i in range(n): #En este bucle se extraen las distintas porciones en las que dividimos la secuencia
        if i == n - 1:
            porciones.append(sec[porcion_len * i:]) #Si es la porción, se coge hasta el final.
        else:
            porciones.append(sec[porcion_len * i:porcion_len * i + porcion_len])
    return porciones
        

def imprime_salva_informe(sec_nombres, freqs, alfabeto, fichero_destino, imprime=True):
    """Salva los datos formateados en el fichero de destino.
    Da la posibilidad de imprimirlos por pantalla. 
    """
    lineas = []
    fd = open(fichero_destino, "w")
    for i,freqs_secuencia in enumerate(freqs):
        if imprime:
            print(sec_nombres[i])
        lineas.append(sec_nombres[i] + '\n')
        
        for j, freqs_nucl in enumerate(freqs_secuencia):
            freqs_nucl_print = ["%.4f%%" % freq for freq in freqs_nucl]
            print(alfabeto[j] + ": " + str(freqs_nucl_print))
            lineas.append(alfabeto[j] + ": " + str(freqs_nucl_print) + '\n')
        
        if imprime:
            print()
            
        lineas.append('\n')
    fd.writelines(lineas)
    fd.close()

def informe_nucleotidos(archivo_fasta, porciones, alfabeto):
     """Salva los datos formateados en el fichero de destino.
    Da la posibilidad de imprimirlos por pantalla. 
    """
    fd = open(archivo_fasta, "r")
    covid_secs = fd.readlines() # Saca todas las líneas del fichero a una lista
    sec_names = [] # Aquí se almacenan los nombres de las secuencias
    secs = [] # Y aquí las secuencias
    freqs = [] # Aquí las frecuencias de los elementos del alfabeto en cada secuencia
    i = 0
    while(True): #Bucle infinito en el que se extraen las secuencias y sus nombres.
        sec_name = covid_secs[i].strip(">").strip("\n") #Asumimos que el fichero empieza por un nombre
        sec = ''#Aquí vamos componiendo cada secuencia
        i+=1 # Siguiente línea (empieza la secuencia)
        while(i < len(covid_secs) and covid_secs[i][0] != '>'): #Mientras no encontremos un nombre de secuencia...
            sec += covid_secs[i].strip('\n') # ...añadimos la linea sin el salto de línea a la secuencia actual
            i+=1 #Y pasamos a la siguiente línea
        sec = sec.upper() #Finalmente, pasamos toda la secuencia a mayúsculas.
        
        sec_names.append(sec_name) #Se añade el nombre de la secuencia a la lista de nombres...
        secs.append(sec) #Y la secuencia a la lista de secuencias
        
        partes = parte_secuencia(sec, porciones) #Una vez compuesta la secuencia, la partimos
        
        freqs_secuencia = [] #Y calculamos las frecuencias de cada letra en este bucle
        for n in alfabeto:
            freqs_nucleotido = []
            for parte in partes:
                freqs_nucleotido.append(calcula_frecuencia(n, parte))
            freqs_secuencia.append(freqs_nucleotido)
        
        freqs.append(freqs_secuencia)
        
        if i == len(covid_secs):
            break
    
    fichero_destino = 'informe_%s.txt' % archivo_fasta.split("/")[-1] #Finalmente, se salva e imprime todo a un fichero
    imprime_salva_informe(sec_names, freqs, alfabeto, fichero_destino)
    fd.close()
    return freqs

In [ ]:
#Primer caso: origen de replicación de vc. alfabeto de 4 letras.
informe_nucleotidos('../labs/oric.fasta', 4, ('A', 'C', 'T', 'G'))

In [ ]:
#Probamos ahora con un alfabeto de 5 letras
informe_nucleotidos('../data/covid-samples.fasta', 4, ('A', 'C', 'T', 'G', 'N'))

## Partiendo las cadenas
Se reduce todo a un problema de redondeos y de cómo crear trozos lo más iguales posible. 

In [ ]:
#Problemilla :) ver https://docs.python.org/3/tutorial/floatingpoint.html
print(round(2.50000), round(3.50000))

In [ ]:
def parte_secuencia(sec, n, redondeo='floor'):
    porciones = []
    print (len(sec), n)
    if redondeo == 'floor':
        porcion_len = int(len(sec) / n) # Redondeamos al entero que resulta de eliminar la parte decimal.
    else:
        porcion_len = int(1.0 * len(sec) / n + 0.5) # Redondeamos al entero más cercano (intervalo cerrado)
        # porcion_len = round(len(sec) / n) # Podría usar esto, pero también tendría problemas...
    print("porcion_len es %d" % porcion_len)
    print()
    for i in range(n):
        if i == n - 1:
            porciones.append(sec[porcion_len * i:])
        else:
            porciones.append(sec[porcion_len * i:porcion_len * i + porcion_len])
    return porciones
        

In [ ]:
#Ejemplo de uso de los dos tipos de redondeo
cadena = "abcdefghijk"
print("Cadena tiene %d caracteres" % len(cadena))
print()
for parte in parte_secuencia(cadena, 4, 'floor'):
    print(len(parte), parte)
print()
for parte in parte_secuencia(cadena, 4, 'closest'):
    print(len(parte), parte)

In [ ]:
n=4
for i in range(5, 26):
    print("%d (%f) A: %d, B: %d, C:%d" %(i, i / n, int(i / n), int(i / n + 0.5), round(i/n)))

# Problema 2 (5 puntos)

Considera la secuencia `CGATATATCCATAG`. 

1. ¿Cuántas veces aparece el patrón 'ATA'? 
2. ¿Qué devuelve la función count de esa cadena? ¿Ves algún problema?
3. Si respondiste "sí", ¿por qué crees que ocurre esto? Razona y explica una posible solución para este problema.


In [ ]:
seq = "CGATATATCCATAG"
#1. El patrón ATA aparece 3 veces
#2 
print(seq.count("ATA"))


__Explicación__
1. El patrón ATA aparece 3 veces.
2. `seq.count("ATA")` devuelve 2. Efectivamente, hay un problema.
3. Ocurre porque `count()` no tiene en cuenta ocurrencias solapadas de los patrones, como se da en esta secuencia. Una manera de solventar este problema es lanzar una búsqueda incremental del patrón:

In [ ]:
def cuentaPatron(secuencia, patron):
    contador = 0 # en esta variable guardo el número de veces que encuentro el patrón.
    pos = secuencia.find(patron, 0) # busco el patrón empezando en la posición inicial 0 y guardo la posición en la que aparece.
    indice_comienzo=pos+1  # la siguiente búsqueda la iniciaré a partir de la posición donde empezaba el patrón + 1.
    while (pos != -1): # mientras el patrón se encuentre...
        contador += 1 # aumento el contador por cada vez que se encuentre y...
        pos = secuencia.find(patron, indice_comienzo) # Busco de nuevo el patrón esta vez en una posición más a la derecha.
        indice_comienzo = pos+1 # Guardo el índice de la ocurrencia + 1 para empezar a partir de ahí la próxima búsqueda.
    return contador # Finalmente, devuelvo el contador :) 

In [ ]:
cuentaPatron("CGATATATCCATAG", "ATA")